In [1]:
import torch, os
import matplotlib.pyplot as plt
import numpy as np
from source.net import *
from source.utils import *
from source.dataset import *
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from pathlib import Path
from torch.utils.data import Subset

device = "cuda:0"

In [2]:
backbone = load_net('simclr')
fc_head = load_net('fc_head', {'num_classes': 1139})
checkpoint = Path("/homes/mlugli/checkpoints/head_norm/checkpoint20")
net = CellClassifier(backbone, fc_head).to(device)
load_weights(checkpoint, net, device)
print(f"Loaded checkpoint-{checkpoint}")

Loaded checkpoint-/homes/mlugli/checkpoints/head_norm/checkpoint20


In [3]:
from source.dataset import Rxrx1
dataset = Rxrx1("/work/ai4bio2024/rxrx1","metadata_plate_norm.csv")

In [4]:
batch_size = 64
metadata = dataset.get_metadata()
test_indices = metadata.index[metadata.iloc[:, 3] == 'test'].tolist()
test_dataset = Subset(dataset, test_indices)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=False, collate_fn=simple_collate)

In [7]:
net.eval()
true_labels, pred_labels = [], []
for x_batch, siRNA_batch, metadata in tqdm(test_dataloader):
    x_batch, y_batch = x_batch.to(torch.float).to(device), torch.tensor(siRNA_batch).to(device)
    y_pred = net(x_batch).argmax(dim=1)
    true_labels.append(y_batch)
    pred_labels.append(y_pred)

100%|██████████| 538/538 [03:15<00:00,  2.75it/s]


In [8]:
print(f"Accuracy-{str(checkpoint).split('/')[-1]}:{accuracy_score(torch.cat(true_labels).cpu(), torch.cat(pred_labels).cpu())}")

Accuracy-checkpoint20:0.03546119888475836
